# Create SNSF Awards from Swiss National Science Foundation

Creates SNSF awards from the SNSF Data Portal (Swiss National Science Foundation). ~90K grants.

**Prerequisites:**
- Run `scripts/local/snsf_to_s3.py` to download and upload the data first.

**Data source:** https://data.snf.ch/grants
**S3 location:** `s3a://openalex-ingest/awards/snsf/snsf_projects.parquet`

**SNSF funder:**
- funder_id: 4320320924
- display_name: "Schweizerischer Nationalfonds zur Förderung der Wissenschaftlichen Forschung"
- ROR: https://ror.org/00yjd3n13
- DOI: 10.13039/501100001711

**SNSF Funding Instruments:**
- Project funding
- Careers
- Programmes
- Infrastructure
- Science communication

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.snsf_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/snsf/snsf_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~90K)
SELECT COUNT(*) as total_projects FROM openalex.awards.snsf_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.snsf_raw LIMIT 5;

In [ ]:
%sql
-- Check column names
DESCRIBE openalex.awards.snsf_raw;

## Step 2: Create SNSF Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.snsf_awards
USING delta
AS
WITH
-- Get SNSF funder from OpenAlex by explicit funder_id
snsf_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320320924  -- Schweizerischer Nationalfonds
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:grant_number
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.grant_number)))) % 9000000000 as id,

        -- Display name = title (prefer English if available, fallback to original)
        COALESCE(g.title_english, g.title) as display_name,

        -- Description = abstract (may include lay summary in English)
        COALESCE(g.lay_summary_en, g.abstract) as description,

        -- Funder info
        f.funder_id,
        g.grant_number as funder_award_id,

        -- Amount in CHF
        TRY_CAST(g.amount AS DOUBLE) as amount,
        'CHF' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map SNSF funding instruments to types
        CASE
            WHEN LOWER(g.funding_instrument_level1) LIKE '%careers%' THEN 'fellowship'
            WHEN LOWER(g.funding_instrument) LIKE '%fellowship%' THEN 'fellowship'
            WHEN LOWER(g.funding_instrument) LIKE '%professorship%' THEN 'fellowship'
            WHEN LOWER(g.funding_instrument) LIKE '%postdoc%' THEN 'fellowship'
            WHEN LOWER(g.funding_instrument) LIKE '%doc.ch%' THEN 'fellowship'
            WHEN LOWER(g.funding_instrument) LIKE '%ambizione%' THEN 'fellowship'
            WHEN LOWER(g.funding_instrument) LIKE '%prima%' THEN 'fellowship'
            WHEN LOWER(g.funding_instrument) LIKE '%eccellenza%' THEN 'fellowship'
            WHEN LOWER(g.funding_instrument_level1) LIKE '%infrastructure%' THEN 'infrastructure'
            WHEN LOWER(g.funding_instrument_level1) LIKE '%programmes%' THEN 'research'
            WHEN LOWER(g.funding_instrument_level1) LIKE '%project funding%' THEN 'research'
            WHEN LOWER(g.funding_instrument) LIKE '%nccr%' THEN 'research'
            WHEN LOWER(g.funding_instrument) LIKE '%sinergia%' THEN 'research'
            WHEN LOWER(g.funding_instrument) LIKE '%project%' THEN 'research'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = funding instrument name
        g.funding_instrument as funder_scheme,

        -- Provenance
        'snsf' as provenance,

        -- Dates (stored as strings in YYYY-MM-DD format)
        TRY_TO_DATE(g.start_date, 'yyyy-MM-dd') as start_date,
        TRY_TO_DATE(g.end_date, 'yyyy-MM-dd') as end_date,
        YEAR(TRY_TO_DATE(g.start_date, 'yyyy-MM-dd')) as start_year,
        YEAR(TRY_TO_DATE(g.end_date, 'yyyy-MM-dd')) as end_year,

        -- Lead investigator - SNSF has responsible_applicant as single name string
        CASE
            WHEN g.responsible_applicant IS NOT NULL AND TRIM(g.responsible_applicant) != '' THEN
                struct(
                    -- Try to extract first name (everything before last space)
                    CASE 
                        WHEN INSTR(TRIM(g.responsible_applicant), ' ') > 0 
                        THEN TRIM(SUBSTRING(TRIM(g.responsible_applicant), 1, LENGTH(TRIM(g.responsible_applicant)) - INSTR(REVERSE(TRIM(g.responsible_applicant)), ' ')))
                        ELSE NULL
                    END as given_name,
                    -- Try to extract last name (everything after last space)
                    CASE 
                        WHEN INSTR(TRIM(g.responsible_applicant), ' ') > 0 
                        THEN TRIM(SUBSTRING(TRIM(g.responsible_applicant), LENGTH(TRIM(g.responsible_applicant)) - INSTR(REVERSE(TRIM(g.responsible_applicant)), ' ') + 2))
                        ELSE g.responsible_applicant
                    END as family_name,
                    CAST(NULL AS STRING) as orcid,
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        COALESCE(g.research_institution, g.institute) as name,
                        COALESCE(g.institute_country, 'Switzerland') as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not available in SNSF data)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL - construct from grant number
        CONCAT('https://data.snf.ch/grants/grant/', g.grant_number) as landing_page_url,

        -- No DOI for SNSF grants typically
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.grant_number)))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.snsf_raw g
    CROSS JOIN snsf_funder f
    WHERE g.grant_number IS NOT NULL
      AND TRIM(g.grant_number) != ''
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Insert into openalex_awards_raw with priority
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    14 as priority
FROM openalex.awards.snsf_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~90K)
SELECT COUNT(*) as total_snsf_awards FROM openalex.awards.snsf_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    currency,
    start_date,
    end_date,
    lead_investigator
FROM openalex.awards.snsf_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be SNSF)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.snsf_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.snsf_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution (top 20 funding instruments)
SELECT funder_scheme, COUNT(*) as cnt,
       ROUND(SUM(amount)/1e6, 2) as total_amount_millions_chf
FROM openalex.awards.snsf_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_abstract,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(end_date) as has_end_date,
    COUNT(lead_investigator) as has_pi,
    ROUND(COUNT(lead_investigator) * 100.0 / COUNT(*), 1) as pct_with_pi,
    ROUND(COUNT(amount) * 100.0 / COUNT(*), 1) as pct_with_amount,
    ROUND(SUM(amount)/1e9, 2) as total_amount_billions_chf
FROM openalex.awards.snsf_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt,
       ROUND(SUM(amount)/1e6, 2) as total_amount_millions_chf
FROM openalex.awards.snsf_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 20;

In [ ]:
%sql
-- Check research institutions (top 20)
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as grant_count,
    ROUND(SUM(amount)/1e6, 1) as total_amount_millions_chf
FROM openalex.awards.snsf_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY total_amount_millions_chf DESC
LIMIT 20;